# Prerequisite

Before this notebook can be run, llc_0001_sociodemo_harmonised_selfreport_jupyter_notebook_vX must be run and process_returned_data.py must be run to return llc_0001_full_harmonised.dta to the database. 

See llc_0001_sociodemo_harmonised_selfreport_jupyter_notebook_vX for more details.

Next, archive the 4 project versioning csv files for llc_0002 by moving them into the archive folder (in the llc_0002 project versioning directory) and into a folder named with today's date.

Finally, run provision_data.py with llc_0002.

# 1. Generate Base Tables

Run stata scripts to generate core statistics. 

In [ ]:
# Import and setup running stata from python
import stata_setup
stata_setup.config('C:/Program Files/Stata17', 'se')

In [ ]:
%%stata
do "S:\LLC_0002\Resource Profile Paper\Syntax\Prerequisite.do"
do "S:\LLC_0002\Resource Profile Paper\Syntax\NHSD_Harmonisation.do"
do "S:\LLC_0002\Resource Profile Paper\Syntax\Table 1 Syntax.do"
do "S:\LLC_0002\Resource Profile Paper\Syntax\Table 2 Syntax.do"
do "S:\LLC_0002\Resource Profile Paper\Syntax\Table 3 and 4 Syntax.do"

# 2. Disclosure Check and Format Tables

In [ ]:
# Import necessary python modules
import pandas as pd
from IPython.display import display
import os
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Alignment, Font
from string import ascii_lowercase as alc
import re

Functions

In [ ]:
def insert_space(sheet):
    # Note, only works on cells with no space before the number
    numerals = ["(i)", "(ii)", "(iii)", "(iv)", "(v)", "(vi)", "(vii)", "(viii)", "(ix)", "(x)"]
    max_row = sheet.max_row
    max_col = sheet.max_column
    for char in alc[:max_col]:
        char = char.upper()
        for row in range(1,max_row+1):
            value = str(sheet[char+str(row)].value)
            # numerals hardfix
            if value.split(" ")[0] in numerals:
                leading = value.split(" ")[0] + " "
                remaining = value.split(" ")[2:]
                value = value.split(" ")[1]
            else:
                leading = ""
                remaining = value.split(" ")[1:]
                value = value.split(" ")[0]
            
            try:
                value = int(value)
            except:
                continue
            new_val = str(value)
            if value >= 10000000:
                new_val = new_val[:-6]+" "+new_val[-6:]
            if value >= 10000:
                new_val = new_val[:-3]+" "+new_val[-3:]
            new_val = leading + new_val + " " + " ".join(remaining)
            try:
                sheet[char+str(row)].value = new_val
            except AttributeError: # can't write to merged cells - skip
                print("skipped "+char+str(row))
                pass

        if char == max_col:
            break
    
    return sheet


def round_percent(sheet):
    max_row = sheet.max_row
    max_col = sheet.max_column
    for char in alc[:max_col]:
        char = char.upper()
        for row in range(1,max_row+1):
            value = str(sheet[char+str(row)].value)
            if "%" in value:
                parts = re.split('[(|)]+', value)
                num = parts[1].replace("%","")
                try:
                    value = float(num)
                except:
                    continue
                new_val = parts[0]+"("+str(round(value, 1))+"%)"+parts[2]
                try: 
                    sheet[char+str(row)].value = new_val
                except AttributeError: # can't write to merged cells - skip
                    print("skipped "+char+str(row))

        if char == max_col:
            break
    
    return sheet


## Table1

In [ ]:
# Load A, set LPS col as index
table1A = pd.read_excel(os.path.join("..","Excel Outputs","Table1A_output.xlsx"), skiprows=1).rename(columns = {"Unnamed: 0": "LPS"})
table1A = table1A.set_index("LPS")

# Load B, set LPS col as index
table1B = pd.read_excel(os.path.join("..","Excel Outputs","Table1B_output.xlsx"), skiprows=1).rename(columns = {"Unnamed: 0": "LPS"})
table1B = table1B.set_index("LPS")

# Get total excluded from A
table1A["# Excluded"] = table1A["# died <2020"]+table1A["# died >=2020"]+table1A["# withdrawn from LPS"]+table1A["# dissented from LLC"]+table1A["# dissented from record linkage"]+table1A["# governance not established"]+table1A["# other"]
# Merge val and % cols
table1A[ "# sent to UK LLC"] = table1A["# sent to UK LLC"].astype(str) +" "+ table1A["percenttotal"].astype(str)
table1A.drop(columns = ["percenttotal"], inplace = True)

# Reformat B to merge '%' into # sent to UK LLC
table1B["# sent to UK LLC"] = table1B["# sent to UK LLC"].astype(str) +" "+ table1B["(%)"].astype(str)
table1B.drop(columns = ["(%)"], inplace = True)

# Merge A & B
table1 = pd.concat([table1B, table1A[["# enrolled", "# sent to UK LLC","# Excluded"]]])
table1["# Excluded"].loc["Total"] = table1["# Excluded"].loc["Total"].astype(str) +" a"
# Rename to final columns
table1.columns = ["N participants enrolled in LPS", "N (%) participants sent to UK LLC", "N participants not sent to UK LLC"]

# Organise exclusions (sub table under main table)
exclusion_table = pd.DataFrame([str(table1A["# died <2020"].values[0])+" excluded because they died <2020", str(table1A["# died >=2020"].values[0])+" excluded because they died >= 2020", str(table1A["# withdrawn from LPS"].values[0])+" excluded because they withdrew from LPS", str(table1A["# dissented from LLC"].values[0])+" excluded because they dissented from UK LLC", str(table1A["# dissented from record linkage"].values[0])+" excluded because they dissented from record linkage", str(table1A["# governance not established"].values[0])+" excluded because governance not established", str(table1A["# other"].values[0])+" excluded for other reasons"], columns = ["a N participants not sent to UK LLC and reasons for exclusion:"], index = None)
numerals = ["(i)", "(ii)", "(iii)", "(iv)", "(v)", "(vi)", "(vii)", "(viii)", "(ix)", "(x)"]
for index, row in exclusion_table.iterrows():
    exclusion_table.iloc[index] = numerals[index] + " " + row["a N participants not sent to UK LLC and reasons for exclusion:"]

# Write to excel file
writer = pd.ExcelWriter(os.path.join("..","Excel Outputs","Table1_processed.xlsx"), engine = "xlsxwriter")
workbook = writer.book
worksheet = workbook.add_worksheet("Sheet1")
writer.sheets["Sheet1"] = worksheet

table1.to_excel(writer, sheet_name = "Sheet1")
exclusion_table.to_excel(writer, sheet_name = "Sheet1",startrow = len(table1)+1,startcol = 0, index = False)

writer.save()

# Reload excel file for styling
workbook = load_workbook(os.path.join("..","Excel Outputs","Table1_processed.xlsx"))
sheet = workbook["Sheet1"]

# Format exclusions sheet
sheet.merge_cells("A22:D22")
sheet.merge_cells("A23:D23")
sheet.merge_cells("A24:D24")
sheet.merge_cells("A25:D25")
sheet.merge_cells("A26:D26")
sheet.merge_cells("A27:D27")
sheet.merge_cells("A28:D28")
sheet.merge_cells("A29:D29")


# Add coloured cells for subheadings
colour = PatternFill(start_color = "D9D9D9", end_color = "D9D9D9", fill_type="solid")
sheet["A1"].fill, sheet["B1"].fill, sheet["C1"].fill, sheet["D1"].fill = colour, colour, colour, colour
sheet["A22"].fill, sheet["B22"].fill, sheet["C22"].fill, sheet["D22"].fill = colour, colour, colour, colour
colour = PatternFill(start_color = "f2f2f2", end_color = "f2f2f2", fill_type="solid")
for num in range(23, 30):
    sheet["A"+str(num)].fill, sheet["B"+str(num)].fill, sheet["C"+str(num)].fill, sheet["D"+str(num)].fill = colour, colour, colour, colour

# Set alignment
for col in ["B", "C", "D"]:
    for num in range(2,22):
        sheet[col+str(num)].alignment = Alignment(horizontal = "right")

# remove borders from index columns except right
border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style=None))
for row in range(1, 22):
    sheet["A"+str(row)].border = border

for col in  ["A", "B", "C", "D"]:
    sheet[col+"1"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style="thin"))

for col in ["A","B", "C", "D"]: 
    sheet[col+"22"].border =  Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style="thin"),
                bottom=Side(style="thin"))
# add spaces
sheet = round_percent(sheet)
sheet = insert_space(sheet)

# Save workbook style
workbook.save(os.path.join("..","Excel Outputs","Table1_processed.xlsx"))
# Reload excel to print
display(pd.read_excel(os.path.join("..","Excel Outputs","Table1_processed.xlsx")))

## Table2

In [ ]:
# Load A, set index
table2A = pd.read_excel(os.path.join("..","Excel Outputs","Table2A_output.xlsx"))

# Load B, set index
table2B = pd.read_excel(os.path.join("..","Excel Outputs","Table2B_output.xlsx"), index_col = "LPS")

# Combine number and percent columns in table 2A
table2A[ "Permission"] = table2A["Permission to Link"].astype(str) +" "+ table2A["Unnamed: 2"].astype(str)
table2A[ "S251"] = table2A["S251"].astype(str) +" "+ table2A["Unnamed: 5"].astype(str)
table2A[ "Consent"] = table2A["Consent"].astype(str) +" "+ table2A["Unnamed: 8"].astype(str)
table2A[ "Total"] = table2A["Total"].astype(str) +" "+ table2A["Unnamed: 10"].astype(str)
table2A.index = ["Total"]

table2A.drop(columns = ["Permission to Link", "Unnamed: 2", "Unnamed: 4", "Unnamed: 5", "Unnamed: 7", "Unnamed: 8", "Unnamed: 10"], inplace = True)

table2 = pd.concat([table2B, table2A])
table2.columns = ["N Participants sent to UK LLC", "Participants with permission to link to NHS Digital", " S251", "Consent", "Total"]

# Write to excel file
writer = pd.ExcelWriter(os.path.join("..","Excel Outputs","Table2_processed.xlsx"), engine = "xlsxwriter")
workbook = writer.book
worksheet = workbook.add_worksheet("Sheet1")
writer.sheets["Sheet1"] = worksheet

# Save formatted table
table2.to_excel(writer, sheet_name = "Sheet1", startrow=1)
writer.save()

# Load table for styling
workbook = load_workbook(os.path.join("..","Excel Outputs","Table2_processed.xlsx"))
sheet = workbook["Sheet1"]

#Setup header merge cells
for col in  ["A", "B", "C"]:
    sheet[col+"1"].value = sheet[col+"2"].value
    sheet.merge_cells(col+"1:"+col+"2")
sheet.merge_cells("D1:F1")
sheet["D1"].value = "Participants successfully linked to NHS Digital and legal basis"


sheet["A1"] = "LPS"
sheet["A1"].font = Font(bold = True)
sheet["A1"].alignment = Alignment(horizontal = "center")

# Add coloured cells for subheadings
colour = PatternFill(start_color = "D9D9D9", end_color = "D9D9D9", fill_type="solid")
for col in  ["A", "B", "C", "D", "E", "F"]:
    sheet[col+"1"].font = Font(bold = True)
    sheet[col+"2"].font = Font(bold = True)
    sheet[col+"1"].fill = colour
    sheet[col+"2"].fill = colour
    sheet[col+"18"].fill = colour

# set content alignment
for col in ["B", "C", "D", "E", "F"]:
    for num in range(3,22):
        sheet[col+str(num)].alignment = Alignment(horizontal = "right")
sheet["D1"].alignment = Alignment(horizontal = "center")

# remove borders from index columns
border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style=None),
                bottom=Side(style=None))
for row in range(1, 19):
    sheet["A"+str(row)].border = border
for col in  ["A", "B", "C", "D", "E", "F"]:
    sheet[col+"1"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style=None))
    sheet[col+"2"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style="thin"))

sheet["D1"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style="thin"))
sheet["E1"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style="thin"))
sheet["F1"].border =  Border(left=Side(style=None),
                    right=Side(style=None),
                    top=Side(style=None),
                    bottom=Side(style="thin"))
# add spaces
sheet = insert_space(sheet)
# Save workbook style
workbook.save(os.path.join("..","Excel Outputs","Table2_processed.xlsx"))
# Reload excel to print
display(pd.read_excel(os.path.join("..","Excel Outputs","Table2_processed.xlsx")))

## Table 3

In [ ]:
# Load excel table for styling
workbook = load_workbook(os.path.join("..","Excel Outputs","Table3_output.xlsx"))
sheet = workbook["Sheet1"]

sheet.unmerge_cells("B1:C1")
sheet.unmerge_cells("B2:C2")

# Remove "value" and "source" cells
sheet.delete_rows(1)
sheet.delete_rows(1)

# Disclosure check process on table 3
for col in ["B","C"]:
    sub_section_count = 0
    for row in range(3,50):
        cell_content = sheet[col+str(row)].value

        if sheet["A"+str(row)].value == "  Missing":
            if cell_content == "":
                cell_content = 0
            # Add count of concealed participants to missing count
            cell_content += sub_section_count
            # check missing itself isn't now less than 10
            if cell_content == 0:
                cell_content = ""
            elif cell_content < 10:
                cell_content = "<10"
            sheet[col+str(row)].value = cell_content
            sub_section_count = 0

        # No action if row is empty or "total"
        elif cell_content == None or cell_content == "" or sheet["A"+str(row)].value == "Total":
            continue

        else:
            if cell_content < 10:
                sub_section_count += sheet[col+str(row)].value
                sheet[col+str(row)].value = "<10"

# Add heading to cell A1
sheet["A1"].value = "Sociodemographic variables"
# Rename B1, C1
sheet["B1"].value = "LPS collected self-reported"
sheet["C1"].value = "NHS Digital routine health records"

# Add total to column headers
sheet["B1"].value = sheet["B1"].value + " (n={})".format(sheet["B7"].value)
sheet["C1"].value = sheet["C1"].value + " (n={})".format(sheet["C7"].value)

# Add coloured cells for headings and subheadings
colour = PatternFill(start_color = "D9D9D9", end_color = "D9D9D9", fill_type="solid")
sheet["A1"].fill, sheet["B1"].fill, sheet["C1"].fill = colour, colour, colour
colour = PatternFill(start_color = "f2f2f2", end_color = "f2f2f2", fill_type="solid")
sheet["A2"].fill, sheet["B2"].fill, sheet["C2"].fill = colour, colour, colour
sheet["A8"].fill, sheet["B8"].fill, sheet["C8"].fill = colour, colour, colour
sheet["A17"].fill, sheet["B17"].fill, sheet["C17"].fill = colour, colour, colour
sheet["A25"].fill, sheet["B25"].fill, sheet["C25"].fill = colour, colour, colour
sheet["A35"].fill, sheet["B35"].fill, sheet["C35"].fill = colour, colour, colour
sheet["A44"].fill, sheet["B44"].fill, sheet["C44"].fill = colour, colour, colour

# Make headers bold
sheet["A1"].font = Font(bold = True)
sheet["B1"].font = Font(bold = True)
sheet["C1"].font = Font(bold = True)

#Clear bottom border
for col in ["A", "B", "C"]:
    for row in range(1,50):
        sheet[col+str(row)].border =  Border(left=Side(style=None),
                        right=Side(style=None),
                        top=Side(style=None),
                        bottom=Side(style=None))
for col in ["A", "B", "C"]:
    sheet[col+"1"].border =  Border(left=Side(style=None),
                        right=Side(style=None),
                        top=Side(style=None),
                        bottom=Side(style="thin"))


# add spaces
sheet = insert_space(sheet)
# save style changes
workbook.save(os.path.join("..","Excel Outputs","Table3_processed.xlsx"))
# Load table for print
display(pd.read_excel(os.path.join("..","Excel Outputs","Table3_processed.xlsx")))

## Table 4

In [ ]:
workbook = load_workbook(os.path.join("..","Excel Outputs","Table4_output.xlsx"))
sheet = workbook["Sheet1"]

sheet.unmerge_cells("B1:T1")
sheet.unmerge_cells("B2:T2")

# Remove "value" and "source" cells
sheet.delete_rows(1)
sheet.delete_rows(1)

# Add heading to cell A1
sheet["A1"].value = "Sociodemographic variables"

# Loop through each column, for each subsection disclosure check and add counts <10 to missing
for col in ["B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"]:
    sub_section_count = 0
    for row in range(3,50):
        cell_content = sheet[col+str(row)].value

        if sheet["A"+str(row)].value == "  Missing":
            if cell_content == "":
                cell_content = 0
            # update missing with removed count
            cell_content += sub_section_count
            # check missing itself isn't now less than 10
            if cell_content == 0:
                cell_content = ""
            elif cell_content < 10:
                cell_content = "<10"
            sheet[col+str(row)].value = cell_content
            # update missing total to match
            sheet["T"+str(row)].value += sub_section_count
            sub_section_count = 0

        # No action if row is empty or "total"
        elif cell_content == None or cell_content == "" or sheet["A"+str(row)].value == "Total":
            continue

        else:
            if cell_content < 10:
                sub_section_count += cell_content
                sheet[col+str(row)].value = "<10"
                sheet["T"+str(row)].value -= cell_content

# Calculate percentages and merge into Total column

total = sheet["T7"].value # could be any total row, pick first for simplicity
for row in range(3, 49):
    if sheet["T"+str(row)].value != "" and sheet["T"+str(row)].value != total:
        sheet["T"+str(row)].value = str(sheet["T"+str(row)].value) + " ("+str(round((sheet["T"+str(row)].value/total) *100, 1))+"%)"

# setup "Total with Data" column 
sheet["U1"].value = "Total with data"
for row in range(3, 50):
    if sheet["T"+str(row)].value != "" and sheet["A"+str(row)] != "  Missing":
        sheet["U"+str(row)].value = sheet["T"+str(row)].value

# Calculate percentage of total with data
counts = []
for row in range(3, 50):
    if sheet["A"+str(row-1)].value == "  Total":
        continue
    elif sheet["A"+str(row)].value == "  Missing":
        missing = str(sheet["U"+str(row)].value).split(" ")[0]
        sheet["U"+str(row)].value = ""
    elif sheet["A"+str(row)].value == "  Total":
        total = int(str(sheet["U"+str(row)].value).split(" ")[0])
        if missing != 0 and missing != "":
            total = int(total) - int(missing)

        for index in range(len(counts)):
            cell ="U"+str(row - (len(counts) - index) - 1)
            cell_count = str(sheet[cell].value).split(" ")[0]

            if cell_count == "":
                cell_count = 0
            sheet[cell].value = str(cell_count) + " (" + str(round((int(cell_count)/int(total)) * 100, 1))+"%)"
        counts = []
        sheet["U"+str(row)].value = int(total)

    else:
        counts.append(sheet["U"+str(row)].value)


# Set formatting
# Add coloured cells for subheadings
header_colour = PatternFill(start_color = "D9D9D9", end_color = "D9D9D9", fill_type="solid")
colour = PatternFill(start_color = "f2f2f2", end_color = "f2f2f2", fill_type="solid")
for char in alc[:21]:
    sheet[char+"1"].fill = header_colour
    sheet[char+"2"].fill = colour
    sheet[char+"8"].fill = colour
    sheet[char+"17"].fill = colour
    sheet[char+"25"].fill = colour
    sheet[char+"35"].fill = colour
    sheet[char+"44"].fill = colour

    # Border and bold header
    sheet[char+"1"].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style= None),
                bottom=Side(style="thin"))
    sheet[char+"1"].font = Font(bold = True)
    sheet[char+"48"].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style= None),
                bottom=Side(style=None))
    sheet[char+"49"].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style=None),
                bottom=Side(style=None))

for row in range(2, 49):
    sheet["A"+str(row)].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style= None),
                bottom=Side(style=None))

sheet["A1"].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style= None),
                bottom=Side(style="thin"))


sheet["A48"].border = Border(left=Side(style=None),
                right=Side(style=None),
                top=Side(style= None),
                bottom=Side(style=None))


# add spaces
sheet = insert_space(sheet)
# Save format and style changes
workbook.save(os.path.join("..","Excel Outputs","Table4_processed.xlsx"))
# Reload and display table
display(pd.read_excel(os.path.join("..","Excel Outputs","Table4_processed.xlsx")))
